In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals
try:
  # The %tensorflow_version magic only works in colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf
import os
import numpy as np
import matplotlib.pyplot as plt
from keras.preprocessing import image
from keras.applications import MobileNetV2

TensorFlow 2.x selected.


Using TensorFlow backend.


In [0]:
tf.__version__

'2.0.0'

In [0]:
#downloading the dataset
#a predesigned dataset from another project on github
!git clone https://github.com/salathegroup/plantvillage_deeplearning_paper_dataset

Cloning into 'plantvillage_deeplearning_paper_dataset'...
remote: Enumerating objects: 163224, done.
remote: Total 163224 (delta 0), reused 0 (delta 0), pack-reused 163224
Receiving objects: 100% (163224/163224), 2.00 GiB | 35.73 MiB/s, done.
Resolving deltas: 100% (97/97), done.
Checking out files: 100% (182401/182401), done.


In [0]:
image_size=224
batch_size=64
generator=image.ImageDataGenerator(horizontal_flip=True,rotation_range=10)

In [0]:
train_generator=generator.flow_from_directory('plantvillage_deeplearning_paper_dataset/raw/color',target_size=(image_size,image_size))

Found 54305 images belonging to 38 classes.


In [0]:
for image_batch,label_batch in train_generator:
  break
image_batch.shape,label_batch.shape

((32, 224, 224, 3), (32, 38))

In [0]:
print (train_generator.class_indices)

labels = '\n'.join(sorted(train_generator.class_indices.keys()))

with open('labels.txt', 'w') as f:
  f.write(labels)

{'Apple___Apple_scab': 0, 'Apple___Black_rot': 1, 'Apple___Cedar_apple_rust': 2, 'Apple___healthy': 3, 'Blueberry___healthy': 4, 'Cherry_(including_sour)___Powdery_mildew': 5, 'Cherry_(including_sour)___healthy': 6, 'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot': 7, 'Corn_(maize)___Common_rust_': 8, 'Corn_(maize)___Northern_Leaf_Blight': 9, 'Corn_(maize)___healthy': 10, 'Grape___Black_rot': 11, 'Grape___Esca_(Black_Measles)': 12, 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)': 13, 'Grape___healthy': 14, 'Orange___Haunglongbing_(Citrus_greening)': 15, 'Peach___Bacterial_spot': 16, 'Peach___healthy': 17, 'Pepper,_bell___Bacterial_spot': 18, 'Pepper,_bell___healthy': 19, 'Potato___Early_blight': 20, 'Potato___Late_blight': 21, 'Potato___healthy': 22, 'Raspberry___healthy': 23, 'Soybean___healthy': 24, 'Squash___Powdery_mildew': 25, 'Strawberry___Leaf_scorch': 26, 'Strawberry___healthy': 27, 'Tomato___Bacterial_spot': 28, 'Tomato___Early_blight': 29, 'Tomato___Late_blight': 30, 'Tomato

In [0]:
!cat labels.txt

Apple___Apple_scab
Apple___Black_rot
Apple___Cedar_apple_rust
Apple___healthy
Blueberry___healthy
Cherry_(including_sour)___Powdery_mildew
Cherry_(including_sour)___healthy
Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot
Corn_(maize)___Common_rust_
Corn_(maize)___Northern_Leaf_Blight
Corn_(maize)___healthy
Grape___Black_rot
Grape___Esca_(Black_Measles)
Grape___Leaf_blight_(Isariopsis_Leaf_Spot)
Grape___healthy
Orange___Haunglongbing_(Citrus_greening)
Peach___Bacterial_spot
Peach___healthy
Pepper,_bell___Bacterial_spot
Pepper,_bell___healthy
Potato___Early_blight
Potato___Late_blight
Potato___healthy
Raspberry___healthy
Soybean___healthy
Squash___Powdery_mildew
Strawberry___Leaf_scorch
Strawberry___healthy
Tomato___Bacterial_spot
Tomato___Early_blight
Tomato___Late_blight
Tomato___Leaf_Mold
Tomato___Septoria_leaf_spot
Tomato___Spider_mites Two-spotted_spider_mite
Tomato___Target_Spot
Tomato___Tomato_Yellow_Leaf_Curl_Virus
Tomato___Tomato_mosaic_virus
Tomato___healthy

In [0]:
IMG_SHAPE = (image_size,image_size, 3)

base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                              include_top=False, 
                                              weights='imagenet')

9412608/9406464 [==============================] - 0s 0us/step


In [0]:
base_model.trainable=False

In [0]:
# model = tf.keras.Sequential([
#   base_model,
#   tf.keras.layers.Conv2D(32, 3, activation='relu'),
#   tf.keras.layers.Dropout(0.2),
#   tf.keras.layers.GlobalAveragePooling2D(),
#   tf.keras.layers.Dense(38, activation='softmax')
# ])

In [0]:
model=tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(400,activation='tanh'),
    tf.keras.layers.Dense(400,activation='tanh'),
    tf.keras.layers.Dense(200,activation='tanh'),
    tf.keras.layers.Dense(100,activation='tanh'),
    tf.keras.layers.Dense(50,activation='tanh'),
    tf.keras.layers.Dense(38,activation='softmax')
])

In [0]:
model.compile(optimizer=tf.keras.optimizers.Adam(1e-5),loss='categorical_crossentropy',metrics=['accuracy'])

In [0]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenetv2_1.00_224 (Model) (None, 7, 7, 1280)        2257984   
_________________________________________________________________
conv2d (Conv2D)              (None, 5, 5, 32)          368672    
_________________________________________________________________
dropout (Dropout)            (None, 5, 5, 32)          0         
_________________________________________________________________
global_average_pooling2d (Gl (None, 32)                0         
_________________________________________________________________
dense (Dense)                (None, 38)                1254      
Total params: 2,627,910
Trainable params: 369,926
Non-trainable params: 2,257,984
_________________________________________________________________


In [0]:
print('Number of trainable variables = {}'.format(len(model.trainable_variables)))

Number of trainable variables = 4


In [0]:
epochs = 30

history = model.fit_generator(train_generator,steps_per_epoch=60, epochs=epochs)

Epoch 1/30
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
60/60 [==============================] - 38s 630ms/step - loss: 3.5536 - accuracy: 0.0714
Epoch 2/30
60/60 [==============================] - 36s 605ms/step - loss: 3.3012 - accuracy: 0.1568
Epoch 3/30
60/60 [==============================] - 35s 585ms/step - loss: 3.1494 - accuracy: 0.2328
Epoch 4/30
60/60 [==============================] - 37s 613ms/step - loss: 2.9556 - accuracy: 0.3042
Epoch 5/30
60/60 [==============================] - 36s 604ms/step - loss: 2.8100 - accuracy: 0.3453
Epoch 6/30
60/60 [==============================] - 34s 570ms/step - loss: 2.6145 - accuracy: 0.4007
Epoch 7/30
60/60 [==============================] - 35s 575ms/step - loss: 2.5199 - accuracy: 0.4141
Epoch 8/30
60/60 [==============================] - 37s 613ms/step - loss: 2.3367 - accuracy: 0.4563
Epoch 9/30
60/60 [==============================] - 41s 677ms/step - loss: 2.2495 - accuracy: 0.47

In [0]:
 #fine tuning onwards
base_model.trainable=True

In [0]:
print("Number of layers in the base model: ", len(base_model.layers))

Number of layers in the base model:  155


In [0]:
fine_tune_at=100
for layer in base_model.layers[:fine_tune_at]:
  layer.trainable=False

In [0]:
model.compile(loss='categorical_crossentropy',optimizer=tf.keras.optimizers.Adam(1e-5),metrics=['accuracy'])

In [0]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenetv2_1.00_224 (Model) (None, 7, 7, 1280)        2257984   
_________________________________________________________________
global_average_pooling2d_3 ( (None, 1280)              0         
_________________________________________________________________
dense_8 (Dense)              (None, 400)               512400    
_________________________________________________________________
dense_9 (Dense)              (None, 400)               160400    
_________________________________________________________________
dense_10 (Dense)             (None, 200)               80200     
_________________________________________________________________
dense_11 (Dense)             (None, 100)               20100     
_________________________________________________________________
dense_12 (Dense)             (None, 50)               

In [0]:
print('Number of trainable variables = {}'.format(len(model.trainable_variables)))

Number of trainable variables = 68


In [0]:
history_fine = model.fit_generator(train_generator,steps_per_epoch=30,epochs=5)

Epoch 1/5
30/30 [==============================] - 18s 609ms/step - loss: 0.5902 - accuracy: 0.8781
Epoch 2/5
30/30 [==============================] - 18s 588ms/step - loss: 0.5544 - accuracy: 0.8885
Epoch 3/5
30/30 [==============================] - 18s 593ms/step - loss: 0.5593 - accuracy: 0.8958
Epoch 4/5
30/30 [==============================] - 18s 587ms/step - loss: 0.5578 - accuracy: 0.8740
Epoch 5/5
30/30 [==============================] - 17s 580ms/step - loss: 0.5647 - accuracy: 0.8896


In [0]:
saved_model_dir = 'save/fine_tuning'
tf.saved_model.save(model, saved_model_dir)

converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
tflite_model = converter.convert()

with open('model.tflite', 'wb') as f:
  f.write(tflite_model)

INFO:tensorflow:Assets written to: save/fine_tuning/assets


In [0]:
from google.colab import files

files.download('model.tflite')
files.download('labels.txt')